In [1]:
pip install -U pyautogen

  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.8
    Uninstalling protobuf-4.25.8:
      Successfully uninstalled protobuf-4.25.8
  Attempting uninstall: pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.35.0 requires pillow<11,>=7.1.0, but you have pillow 12.1.1 which is incompatible.
streamlit 1.35.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.6 which is incompatible.
You should consider upgrading via the 'C:\Users\HP\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import os


In [3]:
import pandas as pd

data = [
    ("2025-07","South","Electronics",120000,1200,25000),
    ("2025-08","South","Electronics",80000,900,22000),
    ("2025-09","South","Electronics",70000,850,20000),

    ("2025-07","South","Grocery",90000,3000,5000),
    ("2025-08","South","Grocery",88000,2900,5200),
    ("2025-09","South","Grocery",87000,2850,5100),

    ("2025-07","North","Electronics",130000,1250,26000),
    ("2025-08","North","Electronics",128000,1230,25500),
    ("2025-09","North","Electronics",131000,1260,26500),
]


In [4]:
df = pd.DataFrame(
    data,
    columns=["month","region","category","revenue","orders","marketing_spend"]
)

In [5]:
df

,month,region,category,revenue,orders,marketing_spend
0,2025-07,South,Electronics,120000,1200,25000
1,2025-08,South,Electronics,80000,900,22000
2,2025-09,South,Electronics,70000,850,20000
3,2025-07,South,Grocery,90000,3000,5000
4,2025-08,South,Grocery,88000,2900,5200
5,2025-09,South,Grocery,87000,2850,5100
6,2025-07,North,Electronics,130000,1250,26000
7,2025-08,North,Electronics,128000,1230,25500
8,2025-09,North,Electronics,131000,1260,26500


In [6]:
import sqlite3

conn = sqlite3.connect("sales.db",check_same_thread=False)
df.to_sql("sales", conn, if_exists="replace", index=False)

pd.read_sql("SELECT * FROM sales", conn)

,month,region,category,revenue,orders,marketing_spend
0,2025-07,South,Electronics,120000,1200,25000
1,2025-08,South,Electronics,80000,900,22000
2,2025-09,South,Electronics,70000,850,20000
3,2025-07,South,Grocery,90000,3000,5000
4,2025-08,South,Grocery,88000,2900,5200
5,2025-09,South,Grocery,87000,2850,5100
6,2025-07,North,Electronics,130000,1250,26000
7,2025-08,North,Electronics,128000,1230,25500
8,2025-09,North,Electronics,131000,1260,26500


In [7]:
def run_sql(query: str) -> str:
    return pd.read_sql(query, conn)

In [8]:
run_sql("SELECT * FROM sales")

,month,region,category,revenue,orders,marketing_spend
0,2025-07,South,Electronics,120000,1200,25000
1,2025-08,South,Electronics,80000,900,22000
2,2025-09,South,Electronics,70000,850,20000
3,2025-07,South,Grocery,90000,3000,5000
4,2025-08,South,Grocery,88000,2900,5200
5,2025-09,South,Grocery,87000,2850,5100
6,2025-07,North,Electronics,130000,1250,26000
7,2025-08,North,Electronics,128000,1230,25500
8,2025-09,North,Electronics,131000,1260,26500


In [13]:
run_sql("SELECT * FROM sales")

,month,region,category,revenue,orders,marketing_spend
0,2025-07,South,Electronics,120000,1200,25000
1,2025-08,South,Electronics,80000,900,22000
2,2025-09,South,Electronics,70000,850,20000
3,2025-07,South,Grocery,90000,3000,5000
4,2025-08,South,Grocery,88000,2900,5200
5,2025-09,South,Grocery,87000,2850,5100
6,2025-07,North,Electronics,130000,1250,26000
7,2025-08,North,Electronics,128000,1230,25500
8,2025-09,North,Electronics,131000,1260,26500


In [14]:
pip install autogen-agentchat

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\HP\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [15]:
import autogen_agentchat

In [18]:
from typing import List, Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console


In [19]:
pip install autogen-ext

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\HP\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [22]:
import sys
print(sys.executable)


C:\Users\HP\AppData\Local\Programs\Python\Python310\python.exe


In [23]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY")
)

In [24]:
business =UserProxyAgent(
    name = "Businessowner",
    description = "You are a business owner who wants to understand the sales performance of your company. You will ask questions about the sales data and the assistant will provide insights based on the data. Your goal is to make informed decisions to improve your business performance."
)

In [25]:
sql_agent = AssistantAgent(
    name = "SqlAgent",
    model_client = model_client,
    description = "You are a helpful assistant that provides insights based on the sales data. You can run SQL queries to retrieve information from the sales database and use that information to answer the business owner's questions. Your goal is to help the business owner understand the sales performance and make informed decisions.",
    system_message = "you are a sql expert of sqllit and you have to query all the time sales database to answer the business owner's question. Always use the run_sql tool to query the sales database and never answer the business owner's question without querying the database. Always use SQL to query the database and never use any other language. Always use the run_sql tool to query the database and never answer the business owner's question without querying the database.and for this database you have sales tables and in sales table you have column month, region, category, revenue, orders and marketing_spend.",
    tools = [run_sql]
)

In [26]:
analyst = AssistantAgent(
    name = "DataAnalyst",
    model_client = model_client,
    description = "You are a helpful assistant that provides insights based on the sales data. You can run SQL queries to retrieve information from the sales database and use that information to answer the business owner's questions. Your goal is to help the business owner understand the sales performance and make informed decisions.",
    system_message = "You are a helpful assistant that provides insights based on the sales data. You can run SQL queries to retrieve information from the sales database and use that information to answer the business owner's questions. Your goal is to help the business owner understand the sales performance and make informed decisions.")

In [27]:
critic = AssistantAgent(
    name = "Critic",
    model_client = model_client,
    description = "You are a critical assistant that evaluates the insights provided by the DataAnalyst. You will analyze the insights and provide feedback on their accuracy, relevance, and usefulness. Your goal is to ensure that the business owner receives high-quality insights that can help them make informed decisions.",
    system_message = "You are a critical assistant that evaluates the insights provided by the DataAnalyst. You will analyze the insights and provide feedback on their accuracy, relevance, and usefulness. Your goal is to ensure that the business owner receives high-quality insights that can help them make informed decisions.")    

In [28]:
termination_condition = MaxMessageTermination(50) | TextMentionTermination("goodbye")

team = SelectorGroupChat(
    participants=[business, sql_agent, analyst, critic],
    model_client=model_client,
    termination_condition=termination_condition,
)

In [ ]:
stream = team.run_stream(task = "why did the revenue dropped in q3 in south region?")
await Console(stream)

---------- TextMessage (user) ----------
why did the revenue dropped in q3 in south region?
---------- TextMessage (DataAnalyst) ----------
To understand why the revenue dropped in Q3 in the South region, we would need to analyze several factors that might have influenced sales performance during that period. Here are some steps we could take to investigate this issue:

1. **Compare Revenue by Month**: We can look at the monthly revenue figures for each month in Q3 to see if there were specific months where sales significantly declined.

2. **Product Performance Analysis**: Determine if specific products or categories experienced a decline in sales, which could indicate issues with those products.

3. **Customer Segments**: Analyze if there were any changes in customer behavior or demographics that could explain the drop in revenue.

4. **Promotional Activities**: Examine the marketing campaigns or promotions that were active during Q3 to see if there were fewer promotions or ineffecti

In [61]:
stream = team.run_stream(task = "give me all the sales data from south")
await Console(stream) 

---------- TextMessage (user) ----------
give me all the sales data from south
---------- TextMessage (DataAnalyst) ----------
Based on the columns available in the sales table, I will provide you with a query to retrieve all sales data specifically for the South region. Here’s the SQL query:

```sql
SELECT *
FROM sales
WHERE region = 'South';
```

This query will return all columns (month, region, category, revenue, orders, and marketing_spend) for the sales in the South region. Would you like me to proceed with running this query to display the results?
---------- ToolCallRequestEvent (SqlAgent) ----------
[FunctionCall(id='call_N8TfcNBaaYqffzTKBOcc26HF', arguments='{"query":"SELECT * FROM sales WHERE region = \'South\';"}', name='run_sql')]
---------- ToolCallExecutionEvent (SqlAgent) ----------
[FunctionExecutionResult(content='     month region     category  revenue  orders  marketing_spend\n0  2025-07  South  Electronics   120000    1200            25000\n1  2025-08  South  Elect

TaskResult(messages=[TextMessage(id='74d817bf-7bba-4f7c-9551-52b4fc252c4d', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 29, 22, 171580, tzinfo=datetime.timezone.utc), content='give me all the sales data from south', type='TextMessage'), TextMessage(id='123aab13-0a24-4f2a-9624-f6f03980c48a', source='DataAnalyst', models_usage=RequestUsage(prompt_tokens=887, completion_tokens=95), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 29, 24, 886952, tzinfo=datetime.timezone.utc), content="Based on the columns available in the sales table, I will provide you with a query to retrieve all sales data specifically for the South region. Here’s the SQL query:\n\n```sql\nSELECT *\nFROM sales\nWHERE region = 'South';\n```\n\nThis query will return all columns (month, region, category, revenue, orders, and marketing_spend) for the sales in the South region. Would you like me to proceed with running this query to display the results?", type='Tex